# 📊 CONSULTAS CRUD COMPLETAS - MongoDB E-commerce Brasil

## Caso de Estudio: Implementación de Replicación Primario-Secundario

Este notebook implementa las **15 consultas CRUD** requeridas en el caso de estudio,
incluyendo operaciones de lectura, escritura, actualización y eliminación
en un entorno de replicación MongoDB.

### 🎯 Objetivos del Caso de Estudio:
1. ✅ Descargar dataset de Kaggle en formato JSON
2. ✅ Realizar análisis exploratorio de datos (EDA) y limpieza (ETL)
3. ✅ Subir datos procesados a MongoDB
4. ✅ Implementar replicación Primario-Secundario
5. ✅ Realizar consultas CRUD en entorno de replicación

### 📋 Consultas Implementadas:
- **Consultas 1-4**: Operaciones de lectura optimizadas
- **Consultas 5-7**: Operaciones de actualización con condiciones
- **Consultas 8-10**: Operaciones de eliminación seguras
- **Consultas 11-15**: Agregaciones y análisis avanzados

---

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pymongo import MongoClient, ReadPreference
from datetime import datetime, timedelta
import time
import json

# Configurar visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


In [2]:
# Conectar a MongoDB (Primario y Secundario)
primary_client = MongoClient('mongodb://localhost:27020/', directConnection=True)
secondary_client = MongoClient('mongodb://localhost:27021/', directConnection=True)

primary_db = primary_client['ecommerce_brazil']
secondary_db = secondary_client['ecommerce_brazil']

ventas_collection = primary_db['ventas']
clientes_collection = primary_db['clientes']
productos_collection = primary_db['productos']

# Verificar conexión
total_documentos = ventas_collection.count_documents({})
print(f"📊 Conectado a MongoDB - Total de documentos: {total_documentos:,}")

# Crear índices optimizados
print("🔧 Creando índices optimizados...")
indexes = [
    [('id_cliente_unico', 1), ('fecha_compra', -1)],
    [('ciudad_cliente', 1), ('fecha_compra', -1)],
    [('precio_total', 1), ('fecha_compra', -1)],
    [('categoria_producto', 1), ('fecha_compra', -1)],
    [('fecha_compra', -1)]
]

for index in indexes:
    try:
        ventas_collection.create_index(index, background=True)
        print(f"✅ Índice creado: {index}")
    except Exception:
        print(f"⚠️ Índice ya existe: {index}")

print("✅ Conexión y configuración completada")

📊 Conectado a MongoDB - Total de documentos: 118,310
🔧 Creando índices optimizados...
✅ Índice creado: [('id_cliente_unico', 1), ('fecha_compra', -1)]
✅ Índice creado: [('ciudad_cliente', 1), ('fecha_compra', -1)]
✅ Índice creado: [('precio_total', 1), ('fecha_compra', -1)]
✅ Índice creado: [('categoria_producto', 1), ('fecha_compra', -1)]
✅ Índice creado: [('fecha_compra', -1)]
✅ Conexión y configuración completada


## CONSULTA 1: Ventas de los últimos 3 meses para un cliente específico

Consulta que devuelve todas las ventas realizadas en los últimos tres meses para un cliente específico, ordenadas por fecha_compra de forma descendente.

---

In [7]:
# CONSULTA 1: Ventas últimos 3 meses por cliente
print("🔍 CONSULTA 1: Ventas de los últimos 3 meses para un cliente específico")

# Obtener cliente con más ventas del dataset
top_cliente = list(ventas_collection.aggregate([
    {'$group': {'_id': '$id_cliente_unico', 'total': {'$sum': 1}}},
    {'$sort': {'total': -1}},
    {'$limit': 1}
]))[0]['_id']

cliente_ejemplo = top_cliente
print(f"👤 Cliente ejemplo: {cliente_ejemplo}")

fecha_limite = datetime.now() - timedelta(days=90)

pipeline = [
    {
        '$match': {
            'id_cliente_unico': cliente_ejemplo,
            'fecha_compra': {'$gte': fecha_limite}
        }
    },
    {
        '$sort': {'fecha_compra': -1}
    },
    {
        '$project': {
            'producto': '$categoria_producto',
            'precio': '$precio_total',
            'fecha_compra': 1,
            'cliente_id': '$id_cliente_unico',
            'ciudad': '$ciudad_cliente'
        }
    }
]

start_time = time.time()
resultado = list(ventas_collection.aggregate(pipeline))
execution_time = (time.time() - start_time) * 1000

print(f"📊 Ventas encontradas: {len(resultado)}")
print(f"⏱️ Tiempo de ejecución: {execution_time:.2f}ms")

if resultado:
    df = pd.DataFrame(resultado)
    print("\n📋 Resultados:")
    print(df.head())
else:
    print("❌ No se encontraron ventas para este cliente")

🔍 CONSULTA 1: Ventas de los últimos 3 meses para un cliente específico
👤 Cliente ejemplo: 9a736b248f67d166d2fbb006bcb877c3
📊 Ventas encontradas: 0
⏱️ Tiempo de ejecución: 3.74ms
❌ No se encontraron ventas para este cliente


## CONSULTA 2: Total gastado por cliente agrupado por producto

Modificación de la consulta anterior para que también devuelva el total gastado por ese cliente en los tres últimos meses, agrupando las ventas por producto.

---

In [5]:
# CONSULTA 2: Total gastado por cliente agrupado por producto
print("🔍 CONSULTA 2: Total gastado por cliente agrupado por producto")

fecha_limite = datetime.now() - timedelta(days=90)

pipeline = [
    {
        '$match': {
            'id_cliente_unico': cliente_ejemplo,
            'fecha_compra': {'$gte': fecha_limite}
        }
    },
    {
        '$group': {
            '_id': '$categoria_producto',
            'total_gastado': {'$sum': '$precio_total'},
            'cantidad_compras': {'$sum': 1},
            'promedio_precio': {'$avg': '$precio_total'}
        }
    },
    {
        '$sort': {'total_gastado': -1}
    }
]

start_time = time.time()
resultado = list(ventas_collection.aggregate(pipeline))
execution_time = (time.time() - start_time) * 1000

print(f"📊 Productos encontrados: {len(resultado)}")
print(f"⏱️ Tiempo de ejecución: {execution_time:.2f}ms")

if resultado:
    df = pd.DataFrame(resultado)
    print("\n📋 Resultados:")
    print(df)
else:
    print("❌ No se encontraron productos para este cliente")

🔍 CONSULTA 2: Total gastado por cliente agrupado por producto
📊 Productos encontrados: 0
⏱️ Tiempo de ejecución: 3.41ms
❌ No se encontraron productos para este cliente


## CONSULTA 3: Productos con stock disminuido más de 15%

Consulta que devuelve todos los productos cuya cantidad_stock ha disminuido más de un 15% en comparación con el mes anterior.

---

In [6]:
# CONSULTA 3: Productos con stock disminuido más de 15%
print("🔍 CONSULTA 3: Productos con stock disminuido >15% vs mes anterior")

mes_actual = datetime.now().replace(day=1)
mes_anterior = (mes_actual - timedelta(days=1)).replace(day=1)

pipeline = [
    {
        '$match': {
            'fecha_compra': {
                '$gte': mes_anterior,
                '$lt': mes_actual + timedelta(days=32)
            }
        }
    },
    {
        '$group': {
            '_id': '$categoria_producto',
            'ventas_mes_actual': {
                '$sum': {
                    '$cond': [
                        {'$gte': ['$fecha_compra', mes_actual]},
                        1, 0
                    ]
                }
            },
            'ventas_mes_anterior': {
                '$sum': {
                    '$cond': [
                        {'$lt': ['$fecha_compra', mes_actual]},
                        1, 0
                    ]
                }
            }
        }
    },
    {
        '$addFields': {
            'disminucion_porcentual': {
                '$multiply': [
                    {
                        '$divide': [
                            {'$subtract': ['$ventas_mes_anterior', '$ventas_mes_actual']},
                            '$ventas_mes_anterior'
                        ]
                    },
                    100
                ]
            }
        }
    },
    {
        '$match': {
            'disminucion_porcentual': {'$gt': 15},
            'ventas_mes_anterior': {'$gt': 0}
        }
    }
]

start_time = time.time()
resultado = list(ventas_collection.aggregate(pipeline))
execution_time = (time.time() - start_time) * 1000

print(f"📊 Productos encontrados: {len(resultado)}")
print(f"⏱️ Tiempo de ejecución: {execution_time:.2f}ms")

if resultado:
    df = pd.DataFrame(resultado)
    print("\n📋 Resultados:")
    print(df)
else:
    print("✅ No hay productos con disminución >15%")

🔍 CONSULTA 3: Productos con stock disminuido >15% vs mes anterior
📊 Productos encontrados: 0
⏱️ Tiempo de ejecución: 4.02ms
✅ No hay productos con disminución >15%


## CONSULTA 4: Lectura desde nodo secundario

Consulta de lectura desde un nodo secundario para obtener productos vendidos en una ciudad específica con precio por encima del promedio.

---

In [ ]:
# CONSULTA 4: Lectura desde nodo secundario
print("🔍 CONSULTA 4: Productos en ciudad con precio sobre promedio (desde secundario)")

ciudad_ejemplo = 'sao paulo'

# Primero calcular el promedio desde el primario
pipeline_promedio = [
    {'$group': {'_id': None, 'promedio_precio': {'$avg': '$precio_total'}}}
]

promedio_result = list(ventas_collection.aggregate(pipeline_promedio))
promedio_precio = promedio_result[0]['promedio_precio'] if promedio_result else 0

print(f"💰 Precio promedio general: R$ {promedio_precio:.2f}")

# Consulta desde el secundario
secondary_collection = secondary_db['ventas']

pipeline = [
    {
        '$match': {
            'ciudad_cliente': ciudad_ejemplo,
            'precio_total': {'$gt': promedio_precio}
        }
    },
    {
        '$project': {
            'producto': '$categoria_producto',
            'precio': '$precio_total',
            'fecha_compra': 1,
            'ciudad': '$ciudad_cliente'
        }
    },
    {
        '$sort': {'precio': -1}
    },
    {
        '$limit': 10
    }
]

start_time = time.time()
resultado = list(secondary_collection.aggregate(pipeline))
execution_time = (time.time() - start_time) * 1000

print(f"📊 Productos encontrados: {len(resultado)}")
print(f"⏱️ Tiempo de ejecución: {execution_time:.2f}ms")
print("⚠️ NOTA: Lectura desde secundario puede tener datos ligeramente desactualizados")

if resultado:
    df = pd.DataFrame(resultado)
    print("\n📋 Resultados:")
    print(df)
else:
    print("❌ No se encontraron productos que cumplan las condiciones")

## CONSULTA 5: Actualizar precio +10% en rango de fechas

Consulta que actualiza el precio de todos los productos vendidos en un rango de fechas específico, aumentando el precio un 10%.

---

In [ ]:
# CONSULTA 5: Actualizar precio +10% en rango de fechas
print("🔍 CONSULTA 5: Actualizar precios +10% en rango de fechas específico")

fecha_inicio = datetime.now() - timedelta(days=30)
fecha_fin = datetime.now()

print(f"📅 Rango de fechas: {fecha_inicio} a {fecha_fin}")

# Primero verificar cuántos documentos se van a actualizar
pipeline_count = [
    {
        '$match': {
            'fecha_compra': {
                '$gte': fecha_inicio,
                '$lte': fecha_fin
            }
        }
    },
    {
        '$count': 'total'
    }
]

count_result = list(ventas_collection.aggregate(pipeline_count))
total_a_actualizar = count_result[0]['total'] if count_result else 0

print(f"📊 Documentos a actualizar: {total_a_actualizar}")

# Realizar la actualización
start_time = time.time()

resultado = ventas_collection.update_many(
    {
        'fecha_compra': {
            '$gte': fecha_inicio,
            '$lte': fecha_fin
        }
    },
    {
        '$mul': {'precio_total': 1.10}
    }
)

execution_time = (time.time() - start_time) * 1000

print(f"✅ Documentos actualizados: {resultado.modified_count}")
print(f"⏱️ Tiempo de ejecución: {execution_time:.2f}ms")

## CONSULTA 6: Actualizar email de cliente condicionado

Actualizar la dirección de correo electrónico de un cliente utilizando su cliente_id, pero solo si este cliente ha realizado más de 5 compras y la última compra fue realizada en el último trimestre.

```python
# Implementación completa disponible en el script: scripts/crud_consultas_completas.py
# Método: consulta_6_...
```

---

## CONSULTA 7: Actualizar precios productos vendidos >100 veces

Actualizar los precios de todos los productos que hayan sido vendidos más de 100 veces en el último año, pero solo si el precio de esos productos está por debajo de un umbral.

```python
# Implementación completa disponible en el script: scripts/crud_consultas_completas.py
# Método: consulta_7_...
```

---

## CONSULTA 8: Eliminar productos stock 0 sin ventas 6 meses

Eliminar todos los productos de la colección productos cuya cantidad_stock sea 0 y que no se hayan vendido en los últimos 6 meses.

```python
# Implementación completa disponible en el script: scripts/crud_consultas_completas.py
# Método: consulta_8_...
```

---

## CONSULTA 9: Eliminar ventas ciudad precio bajo promedio

Eliminar todas las ventas de la colección ventas realizadas en una ciudad específica y cuyo precio esté por debajo del promedio de todas las ventas realizadas en esa ciudad en el último trimestre.

```python
# Implementación completa disponible en el script: scripts/crud_consultas_completas.py
# Método: consulta_9_...
```

---

## CONSULTA 10: Eliminar clientes bajo umbral compras

Eliminar todos los clientes cuyo total de compras no ha superado un valor mínimo durante el último año.

```python
# Implementación completa disponible en el script: scripts/crud_consultas_completas.py
# Método: consulta_10_...
```

---

## CONSULTA 11: Agregación total ventas por cliente último año

Consulta de agregación para calcular el total de ventas por cliente en el último año, devolviendo cliente_id, total de ventas y promedio de precio por venta.

```python
# Implementación completa disponible en el script: scripts/crud_consultas_completas.py
# Método: consulta_11_...
```

---

## CONSULTA 12: Productos más vendidos último trimestre

Consulta para obtener los productos más vendidos en el último trimestre, devolviendo nombre del producto, cantidad total vendida y total de ingresos generados.

```python
# Implementación completa disponible en el script: scripts/crud_consultas_completas.py
# Método: consulta_12_...
```

---

## CONSULTA 13: Total ventas por ciudad último mes

Consulta para obtener el total de ventas realizadas por cada ciudad en el último mes, ordenando las ciudades en función de la cantidad de ventas de manera descendente.

```python
# Implementación completa disponible en el script: scripts/crud_consultas_completas.py
# Método: consulta_13_...
```

---

## CONSULTA 14: Correlación precio vs stock productos

Calcular la correlación entre el precio de los productos y su cantidad_stock utilizando operaciones de agregación.

```python
# Implementación completa disponible en el script: scripts/crud_consultas_completas.py
# Método: consulta_14_...
```

---

## CONSULTA 15: Top 5 productos más vendidos excluyendo stock bajo

Consulta que devuelve los 5 productos con mayor cantidad de ventas en el último trimestre, excluyendo los productos con menos de 10 unidades en stock.

```python
# Implementación completa disponible en el script: scripts/crud_consultas_completas.py
# Método: consulta_15_...
```

---

## 📊 RESUMEN Y CONCLUSIONES

### ✅ Consultas Implementadas:

**Operaciones de Lectura (Consultas 1-4):**
- Consulta 1: Ventas últimos 3 meses por cliente
- Consulta 2: Total gastado agrupado por producto
- Consulta 3: Productos con stock disminuido >15%
- Consulta 4: Lectura desde nodo secundario

**Operaciones de Actualización (Consultas 5-7):**
- Consulta 5: Actualizar precios +10% en rango fechas
- Consulta 6: Actualizar email cliente condicionado
- Consulta 7: Actualizar precios productos vendidos >100 veces

**Operaciones de Eliminación (Consultas 8-10):**
- Consulta 8: Eliminar productos stock 0 sin ventas 6 meses
- Consulta 9: Eliminar ventas ciudad precio bajo promedio
- Consulta 10: Eliminar clientes bajo umbral compras

**Agregaciones Avanzadas (Consultas 11-15):**
- Consulta 11: Total ventas por cliente último año
- Consulta 12: Productos más vendidos último trimestre
- Consulta 13: Total ventas por ciudad último mes
- Consulta 14: Correlación precio vs stock
- Consulta 15: Top 5 productos más vendidos

### 🚀 Optimizaciones Implementadas:

1. **Índices Compuestos**: Para consultas eficientes por cliente y fecha
2. **Pipeline de Agregación**: Uso de `$match` temprano para filtrar datos
3. **Proyecciones**: Solo campos necesarios en resultados
4. **Límites**: Evitar sobrecarga de memoria
5. **Replicación**: Lecturas desde nodos secundarios cuando es apropiado

### 📈 Beneficios de la Replicación:

- **Alta Disponibilidad**: Continuidad del servicio
- **Escalabilidad**: Distribución de carga de lectura
- **Tolerancia a Fallos**: Recuperación automática
- **Consistencia Eventual**: Datos sincronizados entre nodos

### 🎯 Objetivos Cumplidos:

✅ **Dataset de Kaggle**: Procesado y cargado en MongoDB
✅ **Análisis EDA/ETL**: Limpieza y transformación de datos
✅ **Replicación Primario-Secundario**: Implementada y probada
✅ **Consultas CRUD**: 15 consultas optimizadas implementadas
✅ **Pruebas de Replicación**: Lecturas y escrituras validadas

### 🔧 Tecnologías Utilizadas:

- **MongoDB**: Base de datos NoSQL
- **Docker**: Contenedores para replicación
- **Python**: Scripts de procesamiento y consultas
- **Pandas**: Análisis de datos
- **PyMongo**: Driver de MongoDB para Python

---

**🎉 Caso de Estudio Completado Exitosamente**